Olavo Morais Borges Pereira - 11297792 - Projeto 29

**Abordagem Utilizada**

A abordagem adotada consistiu em transformar o conjunto de dados em uma rede complexa e aplicar algoritmos de detecção de comunidades nessa rede. As comunidades identificadas foram utilizadas como clusters. Para realizar a transformação do conjunto de dados em uma rede complexa, foi empregada a estratégia de atribuir a cada entrada do conjunto de dados um nó, conectando todos os nós entre si. O peso das arestas foi determinado com base em uma métrica que mensura a proximidade entre os nós.

Instalando biblioteca necessária para algoritmos de Detecção de Comunidades:

In [ ]:
pip install python-igraph

Importanto Bibliotacas usadas no código

In [ ]:
import numpy as np
import math
import networkx as nx
import community
import igraph as ig
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, load_wine, make_blobs, make_moons
from sklearn.cluster import KMeans, MeanShift, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

**Métricas de Distância**

Definindo funções que medem a distância entre dois nós. Foram implementadas as seguites distâncias: Distância de Minkowski (usada apenas para o cálculo da ditância de Minkowswi inversa e exponencial), Distância de Minkowski Inversa, Distância de Minkowski Exponencial, Fu Metric, Exponencial da Fu Metric e a Exponencial de Tanimoto. Também foi implementada uma função que recebe dois pontos e um código que define qual das métricas implementadas será usada. Foi usado $\alpha=1$ nas funções exponenciais. Para as funções que envolvem a Distância de Minkowski foram usados $p=0.5,p=1,p=2,p=\infty$.

In [ ]:
#Recebe dois pontos e um peso p, retorna a Distância de Minkowski
def minkowski_dist(x,y,p):
  soma=0
  n=len(x)
  maior=0
  for i in range(n):
    if p==float('inf'):#Se o peso é infinito
      maior=max(maior, abs(x[i]-y[i]))
    else:
      soma+=(abs(x[i]-y[i]))**p

  if p == float('inf'):
    return maior
  return (soma**(1/p))

#Calcula a Distância Inversa de Minkowski entre dois pontos
def inverse_minkowski_dist(x,y,p):
  aux=minkowski_dist(x,y,p)
  if aux==0:
      return 1000000000
  return 1/aux

#Calcula a Distância Exponencial de Minkowski entre dois pontos
def exponential_minkowski_dist(x,y,p,alfa):
  return alfa*(math.e**(-1*alfa*minkowski_dist(x,y,p)))

#Calcula a Métrica de Fu entre dois pontos
def fu_metric(x,y):
  return 1-(minkowski_dist(x,y,2))/(np.linalg.norm(x)+np.linalg.norm(y))

#Calcula a Exponencial da Métrica de Fu
def exponential_fu_metric(x,y,alfa):
  return alfa*(math.e**(-1*alfa*(1-fu_metric(x,y))/2))

#Calcula a Exponencial da distância de Tanimoto
def exponential_tanimoto(x,y,alfa):
  x=np.array(x)
  y=np.array(y)
  tanimoto=(np.matmul(x.T,y))/(np.linalg.norm(x)**2+np.linalg.norm(y)**2-np.matmul(x.T,y))
  # print("tanimoto:"+str(tanimoto))
  aux=-1*alfa*((1-tanimoto)/2)
  # print("Expoente:"+str(aux))
  # print("ExpoTanimoto:"+str(alfa*math.e**(aux)))
  return alfa*(math.e**(aux))

#Recebe dois pontos e um código utilizado para definir qual métrica será usada para cálculo da distância entre os pontos
def generic_dist(x,y,code):
  if(code==0):
    return inverse_minkowski_dist(x,y,1)
  if(code==1):
    return inverse_minkowski_dist(x,y,2)
  if(code==2):
    return inverse_minkowski_dist(x,y,float('inf'))
  if(code==3):
    return inverse_minkowski_dist(x,y,0.5)
  if(code==4):
    return exponential_minkowski_dist(x,y,1,1)
  if(code==5):
    return exponential_minkowski_dist(x,y,2,1)
  if(code==6):
    return exponential_minkowski_dist(x,y,float('inf'),1)
  if(code==7):
    return exponential_minkowski_dist(x,y,0.5,1)
  if(code==8):
    return fu_metric(x,y);
  if(code==9):
    return exponential_fu_metric(x,y,1)
  if(code==10):
    return exponential_tanimoto(x,y,1)

#Recebe o código e exibe na tela qual métrica de distância está associada com esse código, usada para facilitar a leitura dos resultados
def ConverterCode(code):
  if(code==0):
    print("Inverse Minkowski, p=1")
  if(code==1):
    print("Inverse Minkowski, p=2")
  if(code==2):
    print("Inverse Minkowski, p=∞")
  if(code==3):
    print("Inverse Minkowski, p=0.5")
  if(code==4):
    print("Exponential Minkowski, p=1")
  if(code==5):
    print("Exponential Minkowski, p=2")
  if(code==6):
    print("Exponential Minkowski, p=∞")
  if(code==7):
    print("Exponential Minkowski, p=0.5")
  if(code==8):
    print("Fu Metric")
  if(code==9):
    print("Exponential Fu Metric")
  if(code==10):
    print("Exponential Tanimoto")

Função usada para normalizar o dataset:

In [ ]:
def standardize_dataset(dataset):
    # Compute the mean and standard deviation along each column
    mean = np.mean(dataset, axis=0)
    std = np.std(dataset, axis=0)

    # Standardize the dataset
    standardized_dataset = (dataset - mean) / std

    return standardized_dataset

**Avaliação do Resultado**

Implementação da função que calcula o índice de Jaccard:

In [ ]:
#Verifica se 2 nós estão na mesma comunidade
def VerificarPar(comunidades,a,b):
  for comunidade in comunidades:
    if(a in comunidade and b in comunidade):
      return True

  return False

#Calcula o índice de Jaccard, recebe as comunidades detectadas pelo algoritmo e os corretos labels de cada nó
def JaccardIndex(comunidades,y):
  a=0 #Número de pares que estão no mesmo cluster e são da mesma classe
  b=0 #Número de pares que não estão no mesmo cluster e são da mesma classe
  c=0 #Número de pares que estão no mesmo cluster mas são de classes diferentes

  #Para cada par de nó
  for i in range(0,len(y)):
    for j in range(0,len(y)):
      if(i!=j):
        aux=VerificarPar(comunidades,i,j)#Verifica se estão na mesma comunidade
        if(y[i]==y[j] and aux==True):#Se estão juntos e pertencem a mesma classe
          a+=1
        elif y[i]==y[j] and aux==False:#Se estão separados mas são da mesma classe
          b+=1
        elif y[i]!=y[j] and aux==True:#Se estão juntos mas são de classes diferentes
          c+=1
  return a/(a+b+c)#Calcula a métrica


**Algoritmos para Detecção de Comunidades**

Foram utilizados os seguintes algoritmos de detecção de comunidades: Fast Greedy com número de classes desconhecido, Fast Greedy com número de classes conhecido, Label Propagation, SpinGlass e Walktrap. Todas as funções recebem o conjunto de dados, criam o grafo e detectam as comunidades. Em seguida, é exibido o número de comunidades encontradas e o índice de Jaccard. O processo é repetido para todas as métricas de distância implementadas anteriormente.

Implementação do Fast Greedy com número desconhecido de classes

In [ ]:
def FastGreedyDesconhecido(X,y):
  G=nx.Graph()

  print("-----FastGreedy, número de classes desconhecido-----")

  for code in range(0,11): #Percorre todas as distâncias implementadas
    ConverterCode(code) #Exibe qual métrica de distância está sendo usada

    #Monta o grafo a partir do dataset
    G.clear()
    for i in range(0,len(X)):
      for j in range(i+1,len(X)):
        G.add_edge(i,j,weight=generic_dist(X[i],X[j],code))

    comunidades=nx.algorithms.community.greedy_modularity_communities(G,weight="weight")
    # print(comunidades)
    print("Número de comunidades: "+str(len(comunidades)))
    print("JaccardIndex:" +str(JaccardIndex(comunidades,y)))
    print("\n")

Implementação do Fast Greedy com número de classes conhecidos

In [ ]:
def FastGreedyConhecido(X,y,k): #Recebe o dataset e o número de classes
  G=nx.Graph()

  print("-----FastGreedy, número de classes conhecido-----")

  for code in range(0,11): #Percorre todas as distâncias implementadas
    ConverterCode(code) #Exibe qual métrica de distância está sendo usada

    #Monta o grafo a partir do dataset
    G.clear()
    for i in range(0,len(X)):
      for j in range(i+1,len(X)):
        G.add_edge(i,j,weight=generic_dist(X[i],X[j],code))

    comunidades=nx.algorithms.community.greedy_modularity_communities(G,weight="weight",cutoff=k,best_n=k)
    #print(comunidades)
    print("Número de comunidades: "+str(len(comunidades)))
    print("JaccardIndex:" +str(JaccardIndex(comunidades,y)))
    print("\n")

Implementação do Label Propagation

In [ ]:
def LabelPropagation(X,y): #Recebe o dataset
  G=nx.Graph()

  print("-----Label Propagation-----")

  for code in range(0,11): #Percorre todas as distâncias implementadas
    ConverterCode(code) #Exibe qual métrica de distância está sendo usada

    #Monta o grafo a partir do dataset
    G.clear()
    for i in range(0,len(X)):
      for j in range(i+1,len(X)):
        G.add_edge(i,j,weight=generic_dist(X[i],X[j],code))

    comunidades=list(nx.algorithms.community.asyn_lpa_communities(G, weight="weight"))
    #print(comunidades)
    print("Número de comunidades: "+str(len(comunidades)))
    print("Jaccard:"+str(JaccardIndex(comunidades,y)))
    print("\n")

Implementação do SpinGlass

In [ ]:
def SpinGlass(X,y): #Recebe o dataset
  G=ig.Graph()

  print("-----Spinglass-----")

  for code in range(0,11):#Percorre todas as distâncias implementadas
    ConverterCode(code)#Exibe qual métrica de distância está sendo usada

    #Monta o grafo a partir do dataset
    G.delete_vertices(range(G.vcount()))
    G.add_vertices(len(X))
    for i in range(0,len(X)):
      for j in range(i+1,len(X)):
          aux=generic_dist(X[i],X[j],code)
          G.add_edge(i,j,weight=aux)

    comunidades = G.community_spinglass(weights="weight")
    comunidades_list = [list(comunidade) for comunidade in comunidades]
    #print(comunidades_list)
    print("Número de comunidades: "+str(len(comunidades_list)))
    print("Jaccard:"+str(JaccardIndex(comunidades_list,y)))
    print("\n")

Implementação da Walktrap

In [ ]:
def Walktrap(X,y): #Recebe o dataset
  G=ig.Graph()

  print("-----Walktrap-----")

  for code in range(0,11):#Percorre todas as distâncias implementadas
    ConverterCode(code)#Exibe qual métrica de distância está sendo usada

    #Monta o grafo a partir do dataset
    G.delete_vertices(range(G.vcount()))
    G.add_vertices(len(X))
    for i in range(0,len(X)):
      for j in range(i+1,len(X)):
          aux=generic_dist(X[i],X[j],code)
          G.add_edge(i,j,weight=aux)

    wtrap = G.community_walktrap(weights="weight",steps = 4)
    clust = wtrap.as_clustering()
    #print(list(clust))
    print("Número de comunidades: "+str(len(clust)))
    print("Jaccard:"+str(JaccardIndex(list(clust),y)))
    print("\n")

**Algoritmos Tradicionais para Detecção de Clusters**

Foram utilizados os seguintes algoritmos de detecção de clusters: KMeans, Mean Shift, DBSCAN, Expectation Maximization(Gaussian Mixture) com número de clusters deconhecido, EM com número de classes conhecido e Agglomerative Clustering. Todas as funções recebem o conjunto de dados e rodam o respectivo algoritmo para detectam os clusters. Em seguida, é exibido o número de comunidades encontradas e o índice de Jaccard.

In [ ]:
#Recebe o dataset e o número de clusters
def Kmeans(X,y,k):
  print("-----KMeans-----")

  #Roda o algoritmo
  kmeans = KMeans(n_init=10,n_clusters=k, random_state=42)
  labels = kmeans.fit_predict(X)

  #Converte o resultado
  clusters = [[] for _ in range(k)]
  for node, label in enumerate(labels):
      clusters[label].append(node)

  #print(clusters)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

In [ ]:
#Recebe o dataset
def Mean_Shift(X,y):
  print("-----Means Shift-----")

  #Roda o algoritmo
  mean_shift = MeanShift()
  mean_shift.fit(X)
  labels = mean_shift.labels_

  #Converte o resultado
  clusters = [[] for _ in range(len(set(labels)))]
  for node, label in enumerate(labels):
    clusters[label].append(node)

  #print(labels)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

In [ ]:
#REcebe o dataset
def func_dbscan(X,y):
  print("-----DBSCAN-----")

  #Roda o algoritmo
  dbscan = DBSCAN()
  dbscan.fit(X)
  labels = dbscan.labels_

  #Converte o resultado
  clusters = [[] for _ in range(len(set(labels)))]
  for node, label in enumerate(labels):
    clusters[label].append(node)

  #print(labels)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

In [ ]:
#Recebe o dataset
def Gaussian_Mixture_Desconhecido(X,y):
  print("-----Gaussian Mixture número de classes não determinado-----")

  #Roda o algoritmo
  gmm = GaussianMixture()
  gmm.fit(X)
  labels = gmm.predict(X)

  #Converte o resultado
  clusters = [[] for _ in range(len(set(labels)))]
  for node, label in enumerate(labels):
    clusters[label].append(node)

  #print(labels)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

In [ ]:
#Recebe o dataset e o número de classes
def Gaussian_Mixture_Conhecido(X,y,k):
  print("-----Gaussian Mixture Número de classes determinado-----")

  #Roda o algoritmo
  gmm = GaussianMixture(n_components=k)
  gmm.fit(X)
  labels = gmm.predict(X)

  #Converte o resultado
  clusters = [[] for _ in range(len(set(labels)))]
  for node, label in enumerate(labels):
    clusters[label].append(node)

  #print(labels)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

In [ ]:
#Recebe o datase
def Agglomerative_Clustering(X,y):
  print("-----Agglomerative Clustering-----")

  #Roda o algoritmo
  agglomerative = AgglomerativeClustering(n_clusters=3)
  agglomerative.fit(X)
  labels = agglomerative.labels_

  #Converte o resultado
  clusters = [[] for _ in range(len(set(labels)))]
  for node, label in enumerate(labels):
    clusters[label].append(node)

  #print(labels)
  print("Número de comunidades: "+str(len(clusters)))
  print("Jaccard Index: "+str(JaccardIndex(clusters,y)))
  print("\n")

**Resultados**

Rodando o teste no Iris Dataset:

In [ ]:
#Carrega o Iris Dataset
iris=load_iris()
X=iris.data
y=iris.target

#Executa todos os algoritmos de detecção de comunidade e de clustering
FastGreedyDesconhecido(X,y)
FastGreedyConhecido(X,y,3)
LabelPropagation(X,y)
SpinGlass(X,y)
Walktrap(X,y)
Kmeans(X,y,3)
Mean_Shift(X,y)
func_dbscan(X,y)
Gaussian_Mixture_Desconhecido(X,y)
Gaussian_Mixture_Conhecido(X,y,3)
Agglomerative_Clustering(X,y)

-----FastGreedy, número de classes desconhecido-----
Inverse Minkowski, p=1
Número de comunidades: 2
JaccardIndex:0.32610478359908884


Inverse Minkowski, p=2
Número de comunidades: 2
JaccardIndex:0.32610478359908884


Inverse Minkowski, p=∞
Número de comunidades: 2
JaccardIndex:0.32610478359908884


Inverse Minkowski, p=0.5
Número de comunidades: 2
JaccardIndex:0.32610478359908884


Exponential Minkowski, p=1
Número de comunidades: 2
JaccardIndex:0.5951417004048583


Exponential Minkowski, p=2
Número de comunidades: 2
JaccardIndex:0.5951417004048583


Exponential Minkowski, p=∞
Número de comunidades: 2
JaccardIndex:0.5872064777327936


Exponential Minkowski, p=0.5
Número de comunidades: 4
JaccardIndex:0.8366981855353949


Fu Metric
Número de comunidades: 2
JaccardIndex:0.5653441295546558


Exponential Fu Metric
Número de comunidades: 2
JaccardIndex:0.5587044534412956


Exponential Tanimoto
Número de comunidades: 2
JaccardIndex:0.5523886639676113


-----FastGreedy, número de classes co

Rodando o teste no Wine dataset:

In [ ]:
#Carrega o Wine Dataset
wine=load_wine()
X=standardize_dataset(wine.data)
y=wine.target

#Executa todos os algoritmos de detecção de comunidade e de clustering
FastGreedyDesconhecido(X,y)
FastGreedyConhecido(X,y,3)
LabelPropagation(X,y)
SpinGlass(X,y)
Walktrap(X,y)
Kmeans(X,y,3)
Mean_Shift(X,y)
func_dbscan(X,y)
Gaussian_Mixture_Desconhecido(X,y)
Gaussian_Mixture_Conhecido(X,y,3)
Agglomerative_Clustering(X,y)

-----FastGreedy, número de classes desconhecido-----
Inverse Minkowski, p=1
Número de comunidades: 3
JaccardIndex:0.9547044743141226


Inverse Minkowski, p=2
Número de comunidades: 3
JaccardIndex:0.5112016293279023


Inverse Minkowski, p=∞
Número de comunidades: 3
JaccardIndex:0.5801684646076548


Inverse Minkowski, p=0.5
Número de comunidades: 3
JaccardIndex:0.9322404371584699


Exponential Minkowski, p=1
Número de comunidades: 9
JaccardIndex:0.5386135057471264


Exponential Minkowski, p=2
Número de comunidades: 3
JaccardIndex:0.8091143594153053


Exponential Minkowski, p=∞
Número de comunidades: 3
JaccardIndex:0.59391771019678


Exponential Minkowski, p=0.5
Número de comunidades: 22
JaccardIndex:0.19776119402985073


Fu Metric
Número de comunidades: 2
JaccardIndex:0.46642952423299244


Exponential Fu Metric
Número de comunidades: 2
JaccardIndex:0.46642952423299244


Exponential Tanimoto
Número de comunidades: 2
JaccardIndex:0.46642952423299244


-----FastGreedy, número de classes con

Rodando o teste no Moons dataset

In [ ]:
#Cria o Moon Dataset
X,y= make_moons(n_samples=200)

#Executa todos os algoritmos de detecção de comunidade e de clustering
FastGreedyDesconhecido(X,y)
FastGreedyConhecido(X,y,2)
LabelPropagation(X,y)
SpinGlass(X,y)
Walktrap(X,y)
Kmeans(X,y,2)
Mean_Shift(X,y)
func_dbscan(X,y)
Gaussian_Mixture_Desconhecido(X,y)
Gaussian_Mixture_Conhecido(X,y,2)
Agglomerative_Clustering(X,y)

-----FastGreedy, número de classes desconhecido-----
Inverse Minkowski, p=1
Número de comunidades: 4
JaccardIndex:0.5533333333333333


Inverse Minkowski, p=2
Número de comunidades: 4
JaccardIndex:0.584040404040404


Inverse Minkowski, p=∞
Número de comunidades: 4
JaccardIndex:0.5193939393939394


Inverse Minkowski, p=0.5
Número de comunidades: 6
JaccardIndex:0.3381818181818182


Exponential Minkowski, p=1
Número de comunidades: 2
JaccardIndex:0.48514851485148514


Exponential Minkowski, p=2
Número de comunidades: 2
JaccardIndex:0.47299509001636664


Exponential Minkowski, p=∞
Número de comunidades: 2
JaccardIndex:0.47299509001636664


Exponential Minkowski, p=0.5
Número de comunidades: 4
JaccardIndex:0.4939365671641791


Fu Metric
Número de comunidades: 2
JaccardIndex:0.526825518831668


Exponential Fu Metric
Número de comunidades: 2
JaccardIndex:0.5193661971830986


Exponential Tanimoto
Número de comunidades: 2
JaccardIndex:0.5193661971830986


-----FastGreedy, número de classes conhe

Rodando os testes no blobs dataset

In [ ]:
#Cria o Blobs Dataset
X,y= make_blobs(n_samples=200)

#Executa todos os algoritmos de detecção de comunidade e de clustering
FastGreedyDesconhecido(X,y)
FastGreedyConhecido(X,y,2)
LabelPropagation(X,y)
SpinGlass(X,y)
Walktrap(X,y)
Kmeans(X,y,2)
Mean_Shift(X,y)
func_dbscan(X,y)
Gaussian_Mixture_Desconhecido(X,y)
Gaussian_Mixture_Conhecido(X,y,2)
Agglomerative_Clustering(X,y)

-----FastGreedy, número de classes desconhecido-----
Inverse Minkowski, p=1
Número de comunidades: 3
JaccardIndex:0.979951763641845


Inverse Minkowski, p=2
Número de comunidades: 3
JaccardIndex:0.979951763641845


Inverse Minkowski, p=∞
Número de comunidades: 3
JaccardIndex:0.979951763641845


Inverse Minkowski, p=0.5
Número de comunidades: 3
JaccardIndex:0.979951763641845


Exponential Minkowski, p=1
Número de comunidades: 3
JaccardIndex:0.979951763641845


Exponential Minkowski, p=2
Número de comunidades: 3
JaccardIndex:0.979951763641845


Exponential Minkowski, p=∞
Número de comunidades: 3
JaccardIndex:0.979951763641845


Exponential Minkowski, p=0.5
Número de comunidades: 3
JaccardIndex:0.979951763641845


Fu Metric
Número de comunidades: 3
JaccardIndex:0.7979084612250441


Exponential Fu Metric
Número de comunidades: 3
JaccardIndex:0.8723162235177022


Exponential Tanimoto
Número de comunidades: 3
JaccardIndex:1.0


-----FastGreedy, número de classes conhecido-----
Inverse Minkow

**Comentários Finais**

Os algoritmos de detecção de comunidades não são perfeitos e podem apresentar erros, porém possuem algumas vantagens. Eles conseguem ter um desempenho satisfatório mesmo quando o número de classes é desconhecido. Além disso, não há uma única métrica de distância que seja sempre superior às outras. Portanto, é importante testar diferentes algoritmos e métricas para cada caso.

Uma abordagem alternativa para construir uma rede complexa a partir de um conjunto de dados é calcular a similaridade entre dois nós e criar uma aresta somente quando essa similaridade excede um determinado limiar. Esse limiar pode ser estabelecido através do treinamento do modelo com os conjuntos de dados disponíveis. Essa abordagem permite construir uma rede personalizada com base nas características dos dados analisados.